## Import Packages

In [1]:
import tensorflow as tf
#old_v = tf.logging.get_verbosity()
#tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.examples.tutorials.mnist import input_data
!pip install tensorboardcolab
from tensorboardcolab import *
tbc = TensorBoardColab()


mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
http://b3d04238.ngrok.io
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updat

## Data Loading Functions

In [0]:
def TRAIN_SIZE(num):
    print ('Total Training Images in Dataset = ' + str(mnist.train.images.shape))
    print ('--------------------------------------------------')
    x_train = mnist.train.images[:num,:]
    print ('x_train Examples Loaded = ' + str(x_train.shape))
    y_train = mnist.train.labels[:num,:]
    print ('y_train Examples Loaded = ' + str(y_train.shape))
    print('')
    return x_train, y_train

def TEST_SIZE(num):
    print ('Total Test Examples in Dataset = ' + str(mnist.test.images.shape))
    print ('--------------------------------------------------')
    x_test = mnist.test.images[:num,:]
    print ('x_test Examples Loaded = ' + str(x_test.shape))
    y_test = mnist.test.labels[:num,:]
    print ('y_test Examples Loaded = ' + str(y_test.shape))
    return x_test, y_test

## Model Parameters

In [3]:
x_train_m2, y_train_m2 = TRAIN_SIZE(30000)
x_test_m2, y_test_m2 = TEST_SIZE(10000)

# Define model parameters
learning_rate_m1 = 0.001
training_epochs_m1 = 100

number_of_inputs_m1 = 784
number_of_outputs_m1 = 10

# define how many nuerons in hidden layer
layer_1_nodes_m2 = 100

Total Training Images in Dataset = (55000, 784)
--------------------------------------------------
x_train Examples Loaded = (30000, 784)
y_train Examples Loaded = (30000, 10)

Total Test Examples in Dataset = (10000, 784)
--------------------------------------------------
x_test Examples Loaded = (10000, 784)
y_test Examples Loaded = (10000, 10)


## The Network

In [0]:
# Input Layer
with tf.variable_scope('Input_Layer', reuse=tf.AUTO_REUSE):
    X = tf.placeholder(tf.float32, shape=(None, number_of_inputs_m1))

# Hidden Layer
with tf.variable_scope('Hidden_Layer',reuse=tf.AUTO_REUSE):
    weights = tf.get_variable("weights1", shape=[number_of_inputs_m1, layer_1_nodes_m2], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes_m2], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)
    
# Output Layer
with tf.variable_scope('Output_Layer',reuse=tf.AUTO_REUSE):
    weights = tf.get_variable("weights2", shape=[layer_1_nodes_m2, number_of_outputs_m1], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[number_of_outputs_m1], initializer=tf.zeros_initializer())
    prediction = tf.nn.softmax(tf.matmul(layer_1_output, weights) + biases)

# define Y and cost function
with tf.variable_scope('Cost_Function',reuse=tf.AUTO_REUSE):
    Y = tf.placeholder(tf.float32, shape=(None, 10))
    
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))
    #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=X, logits=Y))
    
# define optimizer
with tf.variable_scope('Optimizer',reuse=tf.AUTO_REUSE):
  optimizer = tf.train.AdamOptimizer(learning_rate_m1).minimize(cost)

## Output

In [7]:
# Create a summary operation to log the progress of the network
with tf.variable_scope('logging_m2',reuse=tf.AUTO_REUSE):
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()
    
    
# Initialize a session so that we can run TensorFlow operations
with tf.Session() as session:

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())

    # Create log file writers to record training progress.
    # We'll store training and testing log data separately.
    training_writer = tf.summary.FileWriter('./logs/training/m1', session.graph)
    testing_writer = tf.summary.FileWriter('./logs/testing/m1', session.graph)
    #training_writer = summary_writer('.logs/training', session.graph)
    #testing_writer = summary_writer('./logs/testing', session.graph)

    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs_m1):

        # Feed in the training data to our placeholders 'X' and 'Y' and do one step of neural network training
        session.run(optimizer, feed_dict={X: x_train_m2, Y: y_train_m2})

        # Every 5 training steps, log our progress
        if epoch % 5 == 0:
            # Get the current accuracy scores by running the "cost" operation on the training and test data sets
            training_cost, training_summary = session.run([cost, summary], feed_dict={X: x_train_m2, Y: y_train_m2})
            testing_cost, testing_summary = session.run([cost, summary], feed_dict={X: x_test_m2, Y: y_test_m2})
            

            # Write the current training status to the log files (Which we can view with TensorBoard)
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)

            # Print the current training status to the screen
            print("Epoch: {} - Training Cost: {}  Testing Cost: {}".format(epoch, training_cost, testing_cost))

    # Training is now complete!
    
    # Get the final accuracy scores by running the "cost" operation on the training and test data sets
    final_training_cost = session.run(cost, feed_dict={X: x_train_m2, Y: y_train_m2})
    final_testing_cost = session.run(cost, feed_dict={X: x_test_m2, Y: y_test_m2})
    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(prediction, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    final_test_acc = session.run(accuracy, feed_dict={X: x_test_m2, Y: y_test_m2})

    print("Final Training cost: {}".format(final_training_cost))
    print("Final Testing cost: {}".format(final_testing_cost))
    print("Final Testing Accuracy: {}".format(final_test_acc))

    # Now that the neural network is trained, let's use it to make predictions for our test data.
    # Pass in the X testing data and run the "prediciton" operation
    #Y_predicted_ = session.run(prediction, feed_dict={X: x_test})

session.close()

Epoch: 0 - Training Cost: 0.08819606900215149  Testing Cost: 0.08825217932462692
Epoch: 5 - Training Cost: 0.0729975551366806  Testing Cost: 0.07298777252435684
Epoch: 10 - Training Cost: 0.056383293122053146  Testing Cost: 0.05591967701911926
Epoch: 15 - Training Cost: 0.043386563658714294  Testing Cost: 0.04241699352860451
Epoch: 20 - Training Cost: 0.033768221735954285  Testing Cost: 0.03254327550530434
Epoch: 25 - Training Cost: 0.027304062619805336  Testing Cost: 0.02609267085790634
Epoch: 30 - Training Cost: 0.02304927445948124  Testing Cost: 0.021819980815052986
Epoch: 35 - Training Cost: 0.020197071135044098  Testing Cost: 0.01906726323068142
Epoch: 40 - Training Cost: 0.01825116202235222  Testing Cost: 0.017267558723688126
Epoch: 45 - Training Cost: 0.016831545159220695  Testing Cost: 0.015957122668623924
Epoch: 50 - Training Cost: 0.015726836398243904  Testing Cost: 0.01494979951530695
Epoch: 55 - Training Cost: 0.014861992560327053  Testing Cost: 0.01415685098618269
Epoch: 6